In [1]:
from game import Game


n_players = 5
# 7 type is minimax
player_types = [7 for p in range(n_players)]
game = Game(map="USA", player_types=player_types, players_num=len(player_types))
game.start()

In [2]:
game.player_turn
game.json()['state']

{0: {'Montana': 2,
  'Maryland': 3,
  'Indiana': 1,
  'Washington': 2,
  'Kentucky': 3,
  'Michigan': 3,
  'New Hampshire': 2,
  'Missouri': 2,
  'Nebraska': 2},
 1: {'Tennessee': 2,
  'Wyoming': 2,
  'Louisiana': 2,
  'Idaho': 2,
  'Kansas': 2,
  'New York': 3,
  'Pennsylvania': 1,
  'Rhode Island': 2,
  'Mississippi': 1,
  'West Virginia': 1,
  'Iowa': 2},
 2: {'Minnesota': 4,
  'Wisconsin': 3,
  'California': 2,
  'Vermont': 1,
  'Hawaii': 1,
  'Delaware': 1,
  'Oklahoma': 2,
  'New Mexico': 2,
  'Texas': 3,
  'Georgia': 1},
 3: {'Massachusetts': 3,
  'Ohio': 3,
  'Oregon': 2,
  'New Jersey': 2,
  'Illinois': 3,
  'Connecticut': 2,
  'Alabama': 1,
  'South Carolina': 4},
 4: {'Arizona': 2,
  'Colorado': 1,
  'Florida': 2,
  'Maine': 2,
  'Nevada': 1,
  'Virginia': 1,
  'North Dakota': 2,
  'Utah': 4,
  'North Carolina': 2,
  'Arkansas': 2,
  'Alaska': 1},
 -1: {'South Dakota': 0}}

In [3]:
game.players

In [4]:
%matplotlib inline

In [5]:
PLAYER_COLORS = {
    -1: 'lightgray',  # Light gray for neutral
    0: 'lightblue',   
    1: 'lightpink', 
    2: 'lightgreen',
    3: 'lightyellow',
    4: 'lightsalmon',
    5: 'lightcoral',
    6: 'lightcyan',    # Light cyan for player 6
    7: 'mediumpurple',    # Light pink for player 7
}

In [6]:
from collections import defaultdict


def count_edges_between_owners(G, player_colors=PLAYER_COLORS):
    owner_edge_counts = defaultdict(int)

    for u, v in G.edges():
        owner_u = G.nodes[u].get('owner')
        owner_v = G.nodes[v].get('owner')

        if owner_u is not None and owner_v is not None and owner_u != owner_v and -1 not in [owner_u, owner_v]:
            owner_edge_counts[(player_colors[owner_u], player_colors[owner_v])] += 1
            owner_edge_counts[(player_colors[owner_v], player_colors[owner_u])] += 1

    return owner_edge_counts


def remove_duplicates_from_dict(d, filter_by_player=None, player_colors=PLAYER_COLORS):
    sorted_dict = {}
    for key, value in d.items():
        if filter_by_player is not None and player_colors[filter_by_player] not in key:
            continue
        sorted_key = tuple(sorted(key))
        if sorted_key in sorted_dict and sorted_dict[sorted_key] != value:
            raise ValueError("Cannot remove duplicates with different values from a directed graph.")
        sorted_dict[sorted_key] = value
    return sorted_dict


#remove_duplicates_from_dict(count_edges_between_owners(G))

In [7]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr, pearsonr


def get_edges_df(player, shared_edges):
    data = {}
    for edge, n_edges in shared_edges.items():
        edge_other_player = [p for p in edge if p != player][0]
        data[edge_other_player] = n_edges
    
    return pd.Series(data)


def calculate_and_sort_spearman_corr(int_data, float_data):
    float_data = float_data.sort_index()
    for p in float_data.index:
        if p not in int_data:
            int_data[p] = 0
    int_data = int_data.sort_index()
    # Calculate Spearman correlation
    spearman_corr, _ = spearmanr(int_data, float_data)

    return spearman_corr

In [15]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd


def get_owner_and_troops(state_data):
    owner_and_troops = {}
    
    for player, territories in state_data.items():
        for territory, troops in territories.items():
            owner_and_troops[territory] = {'owner': player, 'troops': troops}
    
    return owner_and_troops


def augment_graph_with_state(G, state_data):
    # Define color mapping for players
 
    player_colors = {
        -1: 'gray',  # Neutral color
        0: 'blue',   # Customize player 0's color (adjust as needed)
        1: 'red',    # Customize player 1's color (adjust as needed)
        # Add more player colors as needed
    }

    # Initialize node colors and labels
    node_colors = []
    node_labels = {}

    # Iterate over nodes in the graph
    for node in G.nodes():
        player = state_data.get(node, -1)  # Default to neutral color if not found in state data
        print(player)
        color = player_colors.get(player, 'gray')  # Get player color, default to gray if not found
        troops = state_data[node] if player != -1 else 0  # Troops count (0 for neutral)

        # Assign the color to nodes and label with troops
        node_colors.append(color)
        node_labels[node] = troops

    return node_colors, node_labels


def risk_graph(game, player_colors=PLAYER_COLORS, plot=True):
    owner_and_troops = get_owner_and_troops(game.json()['state'])
    terr_to_id = {}
    id_to_terr = []
    for i, terr in enumerate(game.json()['territories']):
        terr_to_id[terr['name']] = i
        id_to_terr.append(terr['name'])

    territory_neighbors = {}
    for i, terr in enumerate(game.json()['territories']):
        nei = [terr_to_id[adj_terr] for adj_terr in terr['adjacent_territories']]
        territory_neighbors[i] = nei
    # Create a graph
    G = nx.Graph()

    # Add edges based on your data
    for node, edges in territory_neighbors.items():
        for edge in edges:
            G.add_edge(node, edge)

    node_colors = []
    node_labels = {}  # Initialize a dictionary for node labels

    for node in G.nodes():
        terr_name = id_to_terr[node]
        owner_and_n_troops = owner_and_troops[terr_name]
        owner = owner_and_n_troops['owner']
        n_troops = owner_and_n_troops['troops']
        node_colors.append(player_colors[owner])
        node_labels[node] = str(n_troops)
        G.nodes[node]['owner'] = owner

    # Use a different layout algorithm (e.g., Kamada-Kawai)
    pos = nx.kamada_kawai_layout(G)

    # Customize node and edge attributes for better visualization
    node_options = {
        "node_color": "skyblue",
        "node_size": 1200,
        "linewidths": 0.5,
        "edgecolors": "black",
        "node_color": node_colors,
        #"labels": node_labels
    }
    edge_options = {
        "width": 1.0,
        "edge_color": "gray",
    }
    fig = None
    if plot:
        # Create a plot and axis object
        fig, ax = plt.subplots(figsize=(20, 10))

        # Draw nodes and edges on the specified axis
        nx.draw_networkx_nodes(G, pos, ax=ax, **node_options)
        nx.draw_networkx_edges(G, pos, ax=ax, **edge_options)
        nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=15, font_color="black", font_weight="bold", ax=ax)

        # Adjust the plot limits for a better view
        ax.set_xlim(-1.2, 0.8)
        ax.set_ylim(-1, 1)

        ax.set_title("US Risk Symbolic Map")
        ax.axis("off")  # Turn off axis
    
    # Return both the graph object and the axis object
    return G, fig, player_colors


from copy import deepcopy


def step(game):
    current_player = game.player_turn
    game.players[current_player].get_minimax_move(game)


def simulate(game, num_of_simulations=10, step=True, plot=False, player_colors=PLAYER_COLORS, depth=1):
    simulation_results = {}

    current_player = game.player_turn
    playing_player = current_player
    G, fig, player_colors = risk_graph(game, plot=False)
    game_cahce = deepcopy(game)
    turn_utilities = []

    for i in range(num_of_simulations):
        print(f'simulation {i}')
        for d in range(depth):
            print(f'compute for depth {d}')
            if game.players[game.player_turn] is None:
                continue
            print(f'{player_colors[game.player_turn]} plays!')
            
            game.players[game.player_turn].get_minimax_move(game)
            # In the last one we step
            print(len(game.players[game.player_turn].agent.all_utilities_ls[:]))
            turn_utilities += game.players[game.player_turn].agent.all_utilities_ls[:]
            game.players[game.player_turn].agent.all_utilities_ls = []
            print(len(turn_utilities))
        if i != num_of_simulations-1 or not step:
            game = deepcopy(game_cahce)
        
    for utilities in turn_utilities:
        for k, ls in utilities.items():
            if k not in simulation_results:
                simulation_results[k] = []
            simulation_results[k] += ls
        #print(game.players[current_player].agent.all_utilities_ls)
    res_df = pd.DataFrame.from_dict(simulation_results, orient='index')
    #print(f'debug: {simulation_results}')
    #print(res_df.columns)
    res_df = res_df.fillna(res_df.mean())
    res_df = res_df.transpose()
    res_df.rename(columns=player_colors, inplace=True)
    player_corr = res_df.corr().loc[player_colors[playing_player]]

    game.players[playing_player].agent.all_utilities_ls = []
    return playing_player, player_corr


from game import Game


def simulation(num_of_simulations=15, plot=False):
    n_players = 5
    # 7 type is minimax
    player_types = [7 for p in range(n_players)]
    game = Game(map="USA", player_types=player_types, players_num=len(player_types))
    game.start()

    agreements = []

    for i in range(20):
        print(f'step number {i} @@@@@@@@@@@@@@@@@@@@@@@@@@@@')
        G, _, _ = risk_graph(game=game, plot=False)
        shared_edges = remove_duplicates_from_dict(count_edges_between_owners(G=G), filter_by_player=game.player_turn)
        playing_player, player_corr = simulate(game=game, num_of_simulations=num_of_simulations, depth=len(game.players))
        print(f'correlation for player {PLAYER_COLORS[playing_player]}')
        print(player_corr)
        no_na_corr = player_corr[player_corr.index != PLAYER_COLORS[playing_player]].dropna()
        edges_count = get_edges_df(PLAYER_COLORS[playing_player], shared_edges)
        edges_count = edges_count[edges_count.index.isin(no_na_corr.index)]
        agreement = \
            calculate_and_sort_spearman_corr(edges_count, no_na_corr)

        agreements.append(agreement)
        print(f'Consistency {i} @@@@@@@@@@@@@@@@@@@@@@@@@@@@:')
        if agreement is np.nan:
            print('There is an issue.')
        print(edges_count)
        print(no_na_corr)
        print(agreement)
        print('-'*10)
    print(np.mean(agreements))
    return agreements


sim_results = []

for _ in range(20):
    sim_results.append(simulation(num_of_simulations=10))

step number 0 @@@@@@@@@@@@@@@@@@@@@@@@@@@@
simulation 0
compute for depth 0
lightblue plays!
{1: 0.2848685801709123, 2: 0.23422712984189867, 3: 0.10912318253551963, 4: 0.34426516672806184}
{'move_type': 'reinforce', 'territory': 'Maryland', 'troops': 3}
{1: -0.3919088741204608, 2: -0.3433846735700314, 3: -0.41384300100551213, 4: -0.34370047429720996}
{'move_type': 'attack', 'attacking': 'Kansas', 'troops': 3, 'attacked': 'Nebraska', 'attacked_player': 1, 'won': 1, 'probability': 0.15533333333333335}
DICE: [6, 5, 1] [6, 5, 5]
{1: -0.3016434407148886, 2: -0.3674959762649599, 3: -0.5113844544745604, 4: -0.39763843738683335}
{'move_type': 'attack', 'attacking': 'New Hampshire', 'troops': 2, 'attacked': 'Vermont', 'attacked_player': 4, 'won': 0, 'probability': 0.1165}
DICE: [3, 1] [5, 4]
{1: -0.34805647682689805, 2: -0.3383017785717825, 3: -0.5974632126378564, 4: -0.2726850043141329}
{'move_type': 'attack', 'attacking': 'Maryland', 'troops': 3, 'attacked': 'Delaware', 'attacked_player': 3, 

In [14]:
np.mean([r for j in range(len(sim_results)) for r in sim_results[j] if r is not np.nan ])

-0.4600000000000001

In [ ]:
sim_results_2 = sim_results

In [ ]:
player_corr

In [ ]:
get_edges_df(PLAYER_COLORS[playing_player], shared_edges)

In [ ]:
player_corr[player_corr.index != PLAYER_COLORS[playing_player]]

In [ ]:
playing_player

In [ ]:
def step(game):
    current_player = game.player_turn
    game.players[current_player].get_minimax_move(game)

for _ in range(10):
    step(game=game)

In [ ]:
G, _, _ = risk_graph(game=game)

In [ ]:
node_owner = nx.get_node_attributes(G, 'owner')
node_owner
#list(G.nodes())

Explaining actions:

To explain the influence of an attack, we can compute P(attack|success)*utility_for_agents(attack|success) + P(attack|failute)*utility_for_agents(attack|failure)

In [ ]:
simulation_results

 {-1: 'lightgray',
  0: 'lightblue',
  1: 'lightpink',
  2: 'lightgreen',
  3: 'lightyellow',
  4: 'lightsalmon',
  5: 'lightcoral',
  6: 'lightcyan',
  7: 'mediumpurple'}

In [ ]:
import pandas as pd


pd.DataFrame(simulation_results).corr().iloc[current_player]

In [ ]:
current_player

In [ ]:
risk_graph(game)

In [ ]:
for i, terr in enumerate(game.json()['territories']):
    print(terr)
    player = terr['occupying_player']
    
    break

In [ ]:
all_states_risk = set([v for vals in game.json()['state'].values() for v in vals])

In [ ]:
all_states_risk

In [ ]:
territory_neighbors

In [ ]:
# https://medium.com/@jl_ruiz/plot-maps-from-the-us-census-bureau-using-geopandas-and-contextily-in-python-df787647ef77
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import geopandas as gpd 
import os 
from mpl_toolkits.axes_grid1 import make_axes_locatable

path = './us_map/tl_2020_us_state.shp'
df = gpd.read_file(path)
df = df.to_crs("EPSG:4326")

In [ ]:
len(df[df['NAME'].apply(lambda x: x in all_states_risk)])

In [ ]:
len(all_states_risk)

In [ ]:
from shapely.geometry import Point

non_continental = ['VI','MP','GU','PR']#'HI','AS''AK',
us49 = df.copy()
us49 = df[df['NAME'].apply(lambda x: x in all_states_risk)]
for n in non_continental:
    us49 = us49[us49.STUSPS != n]


us49.boundary.plot()
us49['centroid'] = us49.centroid


# New centroid coordinates for Hawaii
new_hawaii_centroid = Point(-130, 35)  # Replace with the desired new coordinates for Hawaii

# New centroid coordinates for Alaska
new_alaska_centroid = Point(-130, 50)  # Replace with the desired new coordinates for Alaska

# Update the geometry of Hawaii (assuming 'NAME' column contains state names)
us49.loc[us49['NAME'] == 'Hawaii', 'centroid'] = new_hawaii_centroid

# Update the geometry of Alaska
us49.loc[us49['NAME'] == 'Alaska', 'centroid'] = new_alaska_centroid


us49 = us49.reset_index()

plt.show()

In [ ]:
us49['centroid'].apply(lambda x: x.x).max()

In [ ]:
len(us49)

In [ ]:
len(us49['centroid'])

In [ ]:
for i in range(len(us49)):
    if i % 2 == 0:
        us49.at[i, 'owner'] = 0
    elif i % 3 == 0:
        us49.at[i, 'owner'] = 1
    elif i % 5 == 0:
        us49.at[i, 'owner'] = 3
    else:
        us49.at[i, 'owner'] = 4

In [ ]:
us49['owner'].unique()

In [ ]:
len(us49['centroid'])

In [ ]:
# Assuming you have a GeoDataFrame of US states called 'us49' with an 'owner' column containing values 1 to 7.

# Create a figure and axis
f, ax = plt.subplots(1, 1, figsize=(8, 8), sharex=True, sharey=True, dpi=300)
plt.title('Map of US States with Centroids')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0, alpha=0.5)

# Replace 'owner' with the name of the column you want to use
plt.ylabel('Owner', fontsize=6)

# Define a list of 7 predefined colors
colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'pink']
colors = ['lightcoral', 'lightblue', 'lightgreen', 'lightseagreen', 'lightpink', 'lightcyan', 'lavender']

#us49.plot(ax=ax, alpha=0.8, cmap='Pastel1', edgecolor='k', legend=True, cax=cax, linewidth=0.1,label='Inline label')

# Iterate through colors and plot centroids with a different color if there are points
for i, color in enumerate(colors):
    subset = us49[us49['owner'] == i]  # Assuming owner values are 1 to 7
    if not subset.empty:
        subset['centroid'].plot(ax=ax, marker='o', color=color, markersize=40, label=f'Owner {i + 1}')

for i in range(len(us49['centroid'])):
    centroid = us49['centroid'].iloc[i]
    ax.annotate("1", xy=(centroid.x, centroid.y), fontsize=8, ha='center')
    ax.annotate(us49['NAME'].iloc[i], xy=(centroid.x, centroid.y), fontsize=2, ha='center')
ax.set_xlim(-150, -60)
# Add a legend
ax.legend(fontsize=6)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import cycle

# Assuming you have a GeoDataFrame of US states called 'us49' with an 'owner' column containing values 1 to 7.

# Create a figure and axis
f, ax = plt.subplots(1, 1, figsize=(8, 8), sharex=True, sharey=True, dpi=300)
plt.title('Map of US States with Centroids')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0, alpha=0.5)

# Replace 'owner' with the name of the column you want to use
plt.ylabel('Owner', fontsize=6)

# Define a list of 7 predefined colors
colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'pink']

# Create a cycle of the 7 colors
color_cycle = cycle(colors)

# Plot the centroids with colors based on the 'owner' column
for owner_value in range(1, 8):
    subset = us49[us49['owner'] == owner_value]
    color = next(color_cycle)  # Get the next color from the cycle
    subset['centroid'].plot(ax=ax, marker='o', color=color, markersize=100, label=f'Owner {owner_value}')

# Add a legend
ax.legend(fontsize=6)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Assuming you have a GeoDataFrame of US states called 'us49' with an 'owner' column containing values 1 to 7 and possibly NaN.

# Define a list of 7 colors for the 7 possible values
colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'pink']

# Create a figure and axis
f, ax = plt.subplots(1, 1, figsize=(8, 8), sharex=True, sharey=True, dpi=300)
plt.title('Map of US States with Centroids')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0, alpha=0.5)

# Replace 'owner' with the name of the column you want to use
plt.ylabel('owner', fontsize=6)

# Fill NaN values in the 'owner' column with a default value, e.g., 0
us49['owner'].fillna(0, inplace=True)

# Plot the centroids with colors based on the 'owner' column values (1 to 7)
for i in range(1, 8):
    subset = us49[us49['owner'] == i]
    color = colors[i - 1]  # Assign a color from the list
    subset['centroid'].plot(ax=ax, marker='o', color=color, markersize=100, label=f'Owner {i}')

# Add a legend
ax.legend(fontsize=6)

plt.show()


In [ ]:

f,ax = plt.subplots(1,1, figsize=(8,8), sharex=True, sharey=True, dpi=300)
plt.title('Map of US States with Centroids')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%",pad=0,alpha=0.5)
us49.plot(ax=ax, alpha=0.5, cmap='Pastel1', edgecolor='k', legend=True, cax=cax, linewidth=0.1,label='Inline label')
plt.ylabel('owner', fontsize=6)
us49['centroid'].plot(ax=ax, marker='o', color='green', markersize=100)

plt.show()

In [ ]:
us49.columns

In [ ]:
f,ax = plt.subplots(1,1, figsize=(8,6), sharex=True, sharey=True, dpi=300)
plt.title('Simple Map of US States')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%",pad=0,alpha=0.5)
us49.plot(ax=ax, alpha=0.5, edgecolor='k', legend=True, cax=cax, linewidth=0.1)
plt.show()

In [ ]:
f,ax = plt.subplots(1,1, figsize=(8,6), sharex=True, sharey=True, dpi=300)
plt.title('Map of US States with Centroids')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%",pad=0,alpha=0.5)
us49.plot('ALAND_miles', ax=ax, alpha=0.5, cmap='Pastel1', edgecolor='k', legend=True, cax=cax, linewidth=0.1,label='Inline label')
plt.ylabel('Square miles', fontsize=6)
us49['centroid'].plot(ax=ax, marker='o', color='red', markersize=5)
plt.show()

In [ ]:
f,ax = plt.subplots(1,1, figsize=(8,6), sharex=True, sharey=True, dpi=300)
plt.title('Simple Map of US States')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%",pad=0,alpha=0.5)
us49.plot('ALAND', ax=ax, alpha=0.5, cmap='Pastel1', edgecolor='k', legend=True, cax=cax, linewidth=0.1)
plt.show()

In [ ]:
territory_neighbors

In [ ]:
territory_neighbors

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
# Create a graph
G = nx.Graph()

# Add edges based on your data
for node, edges in territory_neighbors.items():
    for edge in edges:
        G.add_edge(node, edge)

# Plot the graph
pos = nx.spring_layout(G, seed=42, k=0.5)  # Define the layout for better visualization
plt.figure(figsize=(12, 8))

nx.draw(G, pos, with_labels=True, node_size=1000, node_color='skyblue', font_size=10, font_color='black', font_weight='bold')
plt.title("Your Graph")
plt.show()

In [ ]:
territory_neighbors

In [ ]:
game.json()['territories']